In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
import shap
import optuna
from sklearn.preprocessing import StandardScaler


operating_system ='mac'
if operating_system == 'win':
    path = f'C:/Users/fabau/OneDrive/Documents/GitHub/master-project/'
else:
    path = f'/Users/fabienaugsburger/Documents/GitHub/master-project/'

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#import the X and y data in the format .npy
dataset = 'X_y_EU_1h'

dataset_type = '_all_stat' #all_stat means the 4 stats, 2 means the storms one after another and nothing means only the mean stat

X_train = np.load(f'{path}DATASETS/{dataset}/X_train{dataset_type}.npy')
y_train = np.load(f'{path}DATASETS/{dataset}/y_train{dataset_type}.npy')
X_test = np.load(f'{path}DATASETS/{dataset}/X_test{dataset_type}.npy')
y_test = np.load(f'{path}DATASETS/{dataset}/y_test{dataset_type}.npy')
X_validation = np.load(f'{path}DATASETS/{dataset}/X_validation{dataset_type}.npy')
y_validation = np.load(f'{path}DATASETS/{dataset}/y_validation{dataset_type}.npy')

masking_value = 0#2**0.5

def preprocess_X_data(X):
    # Reshape the X array
    reshape = X.reshape(X.shape[0] * X.shape[1], X.shape[2])
    
    # Fit the StandardScaler to the reshaped X array
    reshape_std = StandardScaler().fit_transform(reshape)

    # Create the mask indicating where NaN values were originally located
    mask_X = np.isnan(X).reshape(X.shape[0], X.shape[1], X.shape[2])
    
    # Replace NaN values with -1e9 in X
    reshape_std[np.isnan(reshape_std)] = masking_value
    
    # Reshape the data back to the original shape for X
    X_processed = reshape_std.reshape(X.shape[0], X.shape[1], X.shape[2])

    return X_processed, mask_X

def preprocess_y_data(y):
    # Reshape the y array

    if dataset_type == '_all_stat':
        reshape = y.reshape(y.shape[0] * y.shape[1], y.shape[2])

        mask_y = np.isnan(y).reshape(y.shape[0], y.shape[1], y.shape[2])

        # Replace NaN values with -1e9 in X
        reshape[np.isnan(reshape)] = masking_value
        
        # Reshape the y array
        y_processed = reshape.reshape(y.shape[0], y.shape[1], y.shape[2])
    else:

    #reshape = y.reshape(y.shape[0] * y.shape[1], 1)

    # Fit the StandardScaler to the y array
    #reshape_std = StandardScaler().fit_transform(reshape)

    # Create the mask indicating where NaN values were originally located
        mask_y = np.isnan(y).reshape(y.shape[0], y.shape[1])

        # Replace NaN values with -1e9 in X
        y[np.isnan(y)] = masking_value
        
        # Reshape the y array
        y_processed = y.reshape(y.shape[0], y.shape[1], 1)
    
    return y_processed, mask_y

X_train, M_X_train = preprocess_X_data(X_train)
X_test, M_X_test = preprocess_X_data(X_test)
X_validation, M_X_validation = preprocess_X_data(X_validation)

'''if dataset_type == '0':
    y_train, M_y_train = preprocess_X_data(y_train)
    y_test, M_y_test = preprocess_X_data(y_test)
    y_validation, M_y_validation = preprocess_X_data(y_validation)'''

y_train, M_y_train = preprocess_y_data(y_train)
y_test, M_y_test = preprocess_y_data(y_test)
y_validation, M_y_validation = preprocess_y_data(y_validation)

/Users/fabienaugsburger/anaconda3/envs/tensor_flow/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num GPUs Available:  1


In [2]:
# Define the custom WeightedMSE class
class WeightedMSE(tf.keras.losses.Loss):
    """ 
    Calculate a weighted MSE. This loss gives you control to weight the 
    pixels that are > 0 differently than the pixels that are 0 in y_true. This
    class is subclassed from tf.keras.lossess.Loss to hopefully enable 
    'stateful'-ness.
 
    weights[0] is the weight for non-zero pixels
    weights[1] is the weight for zero pixels.
    """
    def __init__(self, weights=[1.0, 1.0], name="custom_mse", **kwargs):
        super(WeightedMSE, self).__init__(name=name, **kwargs)
        # store weights
        self.w1 = weights[0]
        self.w2 = weights[1]

    def call(self, y_true, y_pred):
        # build weight_matrix 
        ones_array = tf.ones_like(y_true)
        weights_for_nonzero = tf.math.multiply(ones_array, self.w1)
        weights_for_zero = tf.math.multiply(ones_array, self.w2)
        weight_matrix = tf.where(tf.greater(y_true, 0), weights_for_nonzero, weights_for_zero)
        loss = tf.math.reduce_mean(tf.math.multiply(weight_matrix, tf.math.square(tf.math.subtract(y_pred, y_true))))
        return loss

# Define the model
model_WSE_G = Sequential()

# Define the input shape in the first layer of the neural network
input_shape = (X_train.shape[1], X_train.shape[2])

# Add a Conv1D layer with 32 filters and a kernel size of 3
model_WSE_G.add(Conv1D(filters=32, kernel_size=3, padding='same', input_shape=input_shape))

# Add a Dropout layer
model_WSE_G.add(Dropout(0.1))

# Add the first LSTM layer with 128 units
model_WSE_G.add(LSTM(units=64, return_sequences=True))

# Add a Dropout layer
model_WSE_G.add(Dropout(0.1))

# Add the second LSTM layer with 256 units
model_WSE_G.add(LSTM(units=256, return_sequences=True))

# Add a Dropout layer
model_WSE_G.add(Dropout(0.1))

# Add a Dense (fully connected) layer with 4 units (for 4 stats)
#model_WSE_G.add(Dense(units=128, activation='tanh'))

# Add a Flatten layer
#model_WSE_G.add(Flatten())

# Add the final Dense layer with a linear activation function

#output_length = 185
model_WSE_G.add(Dense(units=4, activation='linear'))

# Compile the model with the custom weighted MSE loss
custom_loss = WeightedMSE(weights=[1.0, 0])
model_WSE_G.compile(optimizer='adam', loss=custom_loss)

# Print the model summary
model_WSE_G.summary()

/Users/fabienaugsburger/anaconda3/envs/tensor_flow/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 185, 32)        │        30,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 185, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 185, 64)        │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 185, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 185, 256)       │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 185, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 185, 4)         │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 384,932 (1.47 MB)

 Trainable params: 384,932 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
run_index = 1 # it should be an integer, e.g. 1

run_logdir = os.path.join(os.curdir, "LSTM_3d_WSE_logs_2/", "run_{:03d}".format(run_index))

print(run_logdir)

# Define callbacks (they can really improve the accuracy if well-chosen!)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model_WSE_G_2.keras", 
                                                   save_best_only=True,
                                                   monitor='loss')
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

./LSTM_3d_WSE_logs_2/run_001


In [5]:
# try to use the fit the model

# Entraîner le modèle
model_WSE_G.fit(X_train, y_train,
            validation_data=(X_test, y_test), 
            epochs=200, 
            verbose=1,  
            batch_size=33,
            callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb],)
          #use_multiprocessing=True)

# Faire des prédictions
y_pred = model_WSE_G.predict(X_test)#.reshape(y_test.shape[0],y_test.shape[1],1)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 101.7034 - val_loss: 113.9364
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 95.2394 - val_loss: 101.9133
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 83.9195 - val_loss: 77.9188
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 64.4692 - val_loss: 58.0436
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 49.0314 - val_loss: 47.2626
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 39.0226 - val_loss: 38.7820
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 32.4321 - val_loss: 32.4140
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 28.1180 - val_loss: 27.9802
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 24.8035 - val_loss: 24.9249
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 22.1155 - val_loss: 22.7505
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 20.4535 - val_loss: 21.1547
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108

In [ ]:
#y_pred = model_Gia_3d.predict(X_test)
'''
value_to_replace = -1e9
new_value = 0

y_test_reshaped = y_test.reshape(y_test.shape[0]*y_test.shape[1])

y_test_0 = [new_value if x == value_to_replace else x for x in y_test_reshaped]
y_test_0 = np.array(y_test_0)

y_test_0 = y_test_0.reshape(y_test.shape[0],y_test.shape[1])
'''
stats = ['max','min','mean','std']
colors = ['red', 'green', 'orange', 'blue']
for storm_n in range(0,len(y_pred)):
    for stat in stats:
        num_stat = stats.index(stat)
                # Find the index where len(y_test[storm_n, :, num_stat] == 0) is zero
        indices = np.where(y_test[storm_n, :, num_stat] == 0)[0]-1
        if len(indices) > 0:
            xlim_index = indices[0]
        else:
            xlim_index = len(y_test[storm_n, :, num_stat])
        color = colors[num_stat % len(colors)]
        plt.plot(y_pred[storm_n, :, num_stat], label=f'y_pred_{stat}', linestyle='--', color=color, alpha=0.7)
        plt.plot(y_test[storm_n, :, num_stat], label=f'y_test_{stat}', color=color)
        plt.xlim(0,xlim_index)
        plt.xlabel('Time steps')
        plt.ylabel('Wind gust speed (m/s)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

# calculate the RMSE

rms = (mean_squared_error(y_test[:,:xlim_index,:].ravel(),y_pred[:,:xlim_index,:].ravel()))**0.5
print(rms) 

In [9]:
# hyper parameter tuning

def create_model(trial):
    model = Sequential()

    # Define the input shape in the first layer of the neural network
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    # Add a Conv1D layer with hyperparameters
    filters = trial.suggest_int('filters', 16, 64)
    kernel_size = trial.suggest_int('kernel_size', 2, 5)
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, padding='same', input_shape=input_shape))

    # Add a Dropout layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    model.add(Dropout(dropout_rate))

    # Add the first LSTM layer with hyperparameters
    lstm_units_1 = trial.suggest_int('lstm_units_1', 32, 128)
    model.add(LSTM(units=lstm_units_1, return_sequences=True))

    # Add a Dropout layer
    model.add(Dropout(dropout_rate))

    # Add the second LSTM layer with hyperparameters
    lstm_units_2 = trial.suggest_int('lstm_units_2', 32, 256)
    model.add(LSTM(units=lstm_units_2, return_sequences=True))

    # Add a Dropout layer
    model.add(Dropout(dropout_rate))

    # Add a Dense (fully connected) layer with 4 units (for 4 stats)
    model.add(Dense(units=4))

    # Compile the model with the custom weighted MSE loss
    custom_loss = WeightedMSE(weights=[1.0, 0])
    model.compile(optimizer='adam', loss=custom_loss)

    return model

def objective(trial):
    model = create_model(trial)
    
    # Fit the model
    history = model.fit(X_train, y_train, 
                        validation_data=(X_test, y_test), 
                        epochs=100, 
                        batch_size=33, 
                        verbose=0)
    
    # Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=1)
    
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

[I 2024-07-12 19:28:19,563] A new study created in memory with name: no-name-3d05ab76-0418-4b58-b1bd-99c243fffa2f


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 8.2783


[I 2024-07-12 19:28:34,643] Trial 0 finished with value: 8.278313636779785 and parameters: {'filters': 52, 'kernel_size': 4, 'dropout_rate': 0.4852181833993443, 'lstm_units_1': 37, 'lstm_units_2': 131}. Best is trial 0 with value: 8.278313636779785.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 4.2415


[I 2024-07-12 19:28:50,421] Trial 1 finished with value: 4.241499900817871 and parameters: {'filters': 42, 'kernel_size': 2, 'dropout_rate': 0.23615421852547658, 'lstm_units_1': 90, 'lstm_units_2': 172}. Best is trial 1 with value: 4.241499900817871.


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.9717


[I 2024-07-12 19:59:13,087] Trial 98 finished with value: 2.9717438220977783 and parameters: {'filters': 28, 'kernel_size': 4, 'dropout_rate': 0.15314692227015406, 'lstm_units_1': 114, 'lstm_units_2': 250}. Best is trial 82 with value: 2.4855475425720215.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3.2175


[I 2024-07-12 19:59:34,511] Trial 99 finished with value: 3.2175352573394775 and parameters: {'filters': 23, 'kernel_size': 3, 'dropout_rate': 0.1770101493212828, 'lstm_units_1': 116, 'lstm_units_2': 224}. Best is trial 82 with value: 2.4855475425720215.


Best hyperparameters:  {'filters': 24, 'kernel_size': 3, 'dropout_rate': 0.10789921318760459, 'lstm_units_1': 122, 'lstm_units_2': 252}
